# Converting to TensorFlow Lite using pytorch_to_keras

In [1]:
import sys

!{sys.executable} -m pip install https://github.com/banoffee-pie/onnx2keras/archive/refs/heads/master.zip
!{sys.executable} -m pip install https://github.com/banoffee-pie/pytorch2keras/archive/refs/heads/master.zip

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     - 59 kB 405 kB/ss


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 54 kB 1.5 MB/s eta 0:00:01


In [2]:
import torch
import io, os, shutil
import tensorflow as tf
import numpy as np
import tflite
from pytorch2keras import pytorch_to_keras
from torch.autograd import Variable

2023-03-28 16:16:00.559302: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def pytorch_to_keras_model(pytorch_model, input_shape) -> tf.keras.Model:
    input_np = np.random.uniform(0, 1, tuple([ 1 ]) + input_shape)
    input_var = Variable(torch.FloatTensor(input_np))

    return pytorch_to_keras(
        pytorch_model,
        input_var,
        [input_shape],
        verbose=True,
        name_policy='renumerate',
        change_ordering=True # change channel_first to channel_last
    )

## Import PyTorch Model
For this example, we use mobilenet_v2.

In [4]:
pytorch_model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
# Switch the model to eval mode
pytorch_model.eval()

Using cache found in /Users/salmankhan/.cache/torch/hub/pytorch_vision_v0.10.0


MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

## Run Inference on PyTorch Model

In [5]:
# Download an image to test against
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

# Download Image Labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

--2023-03-28 16:16:06--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.16’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.001s  

2023-03-28 16:16:06 (18.7 MB/s) - ‘imagenet_classes.txt.16’ saved [10472/10472]



In [6]:
# We will test and train with these params
batch_size = 1
channels = 3
height = 224
width = 224

In [7]:
from PIL import Image
from torchvision import transforms

# Open testing image
input_image = Image.open(filename)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(height),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Note Pytorch is BCHW
input_tensor = preprocess(input_image)

In [8]:
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

with torch.no_grad():
    output = pytorch_model(input_batch)

probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Show top categories per image
vals, idxs = torch.topk(probabilities, 5)
pytorch_results = [(categories[idx], prob) for (idx, prob) in zip(idxs.tolist(), vals.tolist())]
for cat, prob in  pytorch_results:
    print(cat, ':', prob)

Samoyed : 0.8303043246269226
Pomeranian : 0.06988773494958878
keeshond : 0.012964080087840557
collie : 0.010797776281833649
Great Pyrenees : 0.009886783547699451


## Convert to Keras

In [9]:
keras_model = pytorch_to_keras_model(pytorch_model, input_tensor.shape)

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:[torch.Size([3, 224, 224])]
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight classifier.1.weight with shape (1000, 1280).
DEBUG:onnx2keras:Found weight classifier.1.bias with shape (1000,).
DEBUG:onnx2keras:Found weight onnx::Conv_538 with shape (32, 3, 3, 3).
DEBUG:onnx2keras:Found weight onnx::Conv_539 with shape (32,).
DEBUG:onnx2keras:Found weight onnx::Conv_541 with shape (32, 1, 3, 3).
DEBUG:onnx2keras:Found weight onnx::Conv_542 with shape (32,).
DEBUG:onnx2keras:Found weight onnx::Conv_544 with shape (16, 32, 1, 1).
DEBUG:onnx2keras:Found weight onnx::Conv

DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input_0).
DEBUG:onnx2keras:Check input 1 (name onnx::Conv_538).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name onnx::Conv_539).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras.conv:Conv with bias
DEBUG:onnx2keras.conv:2D convolution
DEBUG:onnx2keras.conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Constant
DEBUG:onnx2keras:node_name: onnx::Clip_317
DEBUG:onnx2keras:node_params: {'value': array(0., dtype=float32), 'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are availa

Exported graph: graph(%input_0 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %classifier.1.weight : Float(1000, 1280, strides=[1280, 1], requires_grad=1, device=cpu),
      %classifier.1.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %onnx::Conv_538 : Float(32, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_539 : Float(32, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_541 : Float(32, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_542 : Float(32, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_544 : Float(16, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_545 : Float(16, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_547 : Float(96, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_548 : Float(96, strides=[1], requires_grad=0, device=cpu),
      %onnx

DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Constant
DEBUG:onnx2keras:node_name: onnx::Clip_318
DEBUG:onnx2keras:node_params: {'value': array(6., dtype=float32), 'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: onnx::Conv_319
DEBUG:onnx2keras:node_params: {'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input.4).
DEBUG:onnx2keras:Check input 1 (name onnx::Clip_317).
DEBUG:onnx2keras:Check input 2 (name onnx::Clip_318).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras.clip:Using ReLU(6) instead of clip
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX opera

DEBUG:onnx2keras:node_params: {'value': array(6., dtype=float32), 'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: onnx::Conv_336
DEBUG:onnx2keras:node_params: {'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input.36).
DEBUG:onnx2keras:Check input 1 (name onnx::Clip_334).
DEBUG:onnx2keras:Check input 2 (name onnx::Clip_335).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras.clip:Using ReLU(6) instead of clip
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: input.44
DEBUG:onnx2keras:node_params: {'dilations': [1,

...

DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: onnx::Conv_457
DEBUG:onnx2keras:node_params: {'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input.268).
DEBUG:onnx2keras:Check input 1 (name onnx::Clip_455).
DEBUG:onnx2keras:Check input 2 (name onnx::Clip_456).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras.clip:Using ReLU(6) instead of clip
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: input.276
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 576, 'kernel_shape': [3, 3], 'pads': [1, 1, 1, 1], 'strides': [1, 1], 'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all in

DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Constant
DEBUG:onnx2keras:node_name: onnx::Clip_474
DEBUG:onnx2keras:node_params: {'value': array(6., dtype=float32), 'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: onnx::Conv_475
DEBUG:onnx2keras:node_params: {'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input.300).
DEBUG:onnx2keras:Check input 1 (name onnx::Clip_473).
DEBUG:onnx2keras:Check input 2 (name onnx::Clip_474).
DEBUG:onnx2keras:... found all, cont

DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: input.340
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [1, 1], 'pads': [0, 0, 0, 0], 'strides': [1, 1], 'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input.332).
DEBUG:onnx2keras:Check input 1 (name onnx::Conv_664).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name onnx::Conv_665).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras.conv:Conv with bias
DEBUG:onnx2keras.conv:2D convolution
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:on

DEBUG:onnx2keras:Check input 0 (name input.364).
DEBUG:onnx2keras:Check input 1 (name onnx::Clip_506).
DEBUG:onnx2keras:Check input 2 (name onnx::Clip_507).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras.clip:Using ReLU(6) instead of clip
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: input.372
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 960, 'kernel_shape': [3, 3], 'pads': [1, 1, 1, 1], 'strides': [1, 1], 'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name onnx::Conv_508).
DEBUG:onnx2keras:Check input 1 (name onnx::Conv_676).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name onnx::Conv_677).
DEBUG:onnx2keras:The input not found in layers / mode

DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: onnx::Conv_526
DEBUG:onnx2keras:node_params: {'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input.396).
DEBUG:onnx2keras:Check input 1 (name onnx::Clip_524).
DEBUG:onnx2keras:Check input 2 (name onnx::Clip_525).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras.clip:Using ReLU(6) instead of clip
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: input.404
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [1, 1], 'pads': [0, 0, 0, 0], 'strides': [1, 1], 'change_ordering': True, 'name_polic

### Check keras conversion

In [10]:
def softmax(xs):
    return np.exp(xs)/sum(np.exp(xs))

#transpose the input_batch into BHWC order for tensorflow
tf_input_data = np.transpose( input_batch.numpy(), [0, 2, 3, 1])

keras_output_data = keras_model(tf_input_data)

probs = keras_output_data[0]
data = zip(range(len(probs)), probs)
keras_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  keras_results:
    print(cat, ':', prob)

Samoyed : tf.Tensor(14.355221, shape=(), dtype=float32)
Pomeranian : tf.Tensor(11.880313, shape=(), dtype=float32)
keeshond : tf.Tensor(10.195609, shape=(), dtype=float32)
collie : tf.Tensor(10.012769, shape=(), dtype=float32)
Great Pyrenees : tf.Tensor(9.924629, shape=(), dtype=float32)


## Convert to tflite
Following method from [keras_to_xcore.ipynb](https://colab.research.google.com/github/xmos/ai_tools/blob/develop/docs/notebooks/keras_to_xcore.ipynb)

### Representative Dataset
To convert a model into to a TFLite flatbuffer, a representative dataset is required to help in quantisation. Refer to [Converting a keras model into an xcore optimised tflite model](https://colab.research.google.com/github/xmos/ai_tools/blob/develop/docs/notebooks/keras_to_xcore.ipynb) for more details on this.

In [11]:
# We will use the imagenette dataset(326MB)
!mkdir ./imagenet-dataset
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz -O ./imagenet-dataset/imagenette2-320.tgz
!tar -xf ./imagenet-dataset/imagenette2-320.tgz -C ./imagenet-dataset/

mkdir: ./imagenet-dataset: File exists
--2023-03-28 16:16:16--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.173.224, 52.216.39.32, 52.216.233.29, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.173.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 341663724 (326M) [application/x-tar]
Saving to: ‘./imagenet-dataset/imagenette2-320.tgz’

./imagenet-dataset/ 100%[===================>] 325.83M  5.90MB/s    in 54s     

2023-03-28 16:17:11 (6.02 MB/s) - ‘./imagenet-dataset/imagenette2-320.tgz’ saved [341663724/341663724]



In [12]:
import os
import glob
import random

all_files = glob.glob(os.path.join("./imagenet-dataset/", '**/*.JPEG'), recursive=True)

# Randomly select a subset of images, let's just use 1k for speed
sampled_files = random.sample(all_files, 1000)

def representative_dataset():

    # Iterate over the sampled images and preprocess them
    for image_path in sampled_files:
        pil_image = Image.open(image_path).convert("RGB")
        pytorch_tensor = preprocess(pil_image).unsqueeze(0)
        img_array_np = pytorch_tensor.numpy()

        #swap the axes to BHWC(tf) from BCHW(pytorch)
        img_array_np = np.transpose( img_array_np, [0, 2, 3, 1])
        yield [img_array_np]


### Conversion Process

In [13]:
# Now do the conversion to int8
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32

tflite_int8_model = converter.convert()

# Save the model.
tflite_int8_model_path = 'mobilenet_v2.tflite'
with open(tflite_int8_model_path, 'wb') as f:
  f.write(tflite_int8_model)

INFO:tensorflow:Assets written to: /var/folders/fg/_pf9q2tj3cl9yfjb392zl7rm0000gn/T/tmpj3if3ff1/assets


INFO:tensorflow:Assets written to: /var/folders/fg/_pf9q2tj3cl9yfjb392zl7rm0000gn/T/tmpj3if3ff1/assets
/usr/local/anaconda3/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-03-28 16:17:56.921869: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-03-28 16:17:56.921890: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-03-28 16:17:56.922774: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/fg/_pf9q2tj3cl9yfjb392zl7rm0000gn/T/tmpj3if3ff1
2023-03-28 16:17:56.954269: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-28 16:17:56.954298: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) 

### Run inference

In [14]:
tfl_interpreter = tf.lite.Interpreter(model_path=tflite_int8_model_path)
tfl_interpreter.allocate_tensors()

tfl_input_details = tfl_interpreter.get_input_details()
tfl_output_details = tfl_interpreter.get_output_details()

# Convert PyTorch Input Tensor into Numpy Matrix and Reshape for TensorFlow
tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tf_input_data)
tfl_interpreter.invoke()

tfl_output_data = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

probs = softmax(tfl_output_data[0])
data = zip(range(len(probs)), probs)
tfl_int8_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  tfl_int8_results:
    print(cat, ':', prob)

Samoyed : 0.8701259
Pomeranian : 0.06498451
keeshond : 0.016072603
West Highland white terrier : 0.008832058
collie : 0.0059253285


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Analyse Model

In [15]:
import io
from contextlib import redirect_stdout

def get_operator_counts(model_content):
    with io.StringIO() as buf, redirect_stdout(buf):
        tf.lite.experimental.Analyzer.analyze(model_content=model_content)
        model_structure = buf.getvalue()

    operators = [op.strip().split(" ")[1].split("(")[0] for op in model_structure.split("\n") if "Op#" in op]
    op_counts = {}
    for operator in operators:
        if operator in op_counts:
            op_counts[operator] = op_counts[operator]+1
        else:
            op_counts[operator] = 1

    return (len(operators), op_counts)

def print_operator_counts(model_content):
    total_op_count, op_counts = get_operator_counts(model_content)
    print(f"{'Operator'.upper():<20} {'Count'.upper():>6}")
    print("-"*20 + " " + "-"*6)

    for operator, count in op_counts.items():
        print(f"{operator.lower():<20} {count:>6}")

    print("-"*20 + " " + "-"*6)
    print(f"{'Total'.upper():<20} {total_op_count:>6}")
    print("-"*20 + " " + "-"*6)

# Let's inspect the int8 model
print_operator_counts(tflite_int8_model)


OPERATOR              COUNT
-------------------- ------
quantize                  1
pad                      18
conv_2d                  35
depthwise_conv_2d        17
add                      10
mean                      1
expand_dims               2
transpose                 1
reshape                   1
fully_connected           1
dequantize                1
-------------------- ------
TOTAL                    88
-------------------- ------


### Accuracy

In [34]:
import tensorflow_datasets as tfds
import tensorflow as tf

# Construct a tf.data.Dataset
ds = tfds.load('imagenette', split='train', as_supervised=True, shuffle_files=True)

# Build your input pipeline
ds = ds.shuffle(1000).batch(1).prefetch(10).take(1)
for image, label in ds:
    img = tf.keras.utils.array_to_img(image[0])
    pytorch_batch = preprocess(img).unsqueeze(0)
    tf_batch = np.transpose( pytorch_batch.numpy(), [0, 2, 3, 1])
    
    # use same interpreter as before
    tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tf_batch)
    tfl_interpreter.invoke()

    output = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

    probs = softmax(output[0])
    data = zip(range(len(probs)), probs)
    (idx, prob) = sorted(data, key=lambda x: x[1], reverse=True)[0]
    print(label.numpy())
    print(idx)


2023-03-28 17:03:06.885894: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [16]
	 [[{{node Placeholder/_4}}]]
2023-03-28 17:03:06.886308: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [16]
	 [[{{node Placeholder/_3}}]]


[7]
571
